In [26]:
import pandas as pd
import altair as alt
import numpy as np
from sklearn.decomposition import PCA
import umap
import networkx as nx
import nx_altair as nxa

# Family Portrait

In [41]:
data=pd.read_csv('../public/lang-meta.tsv',sep='\t')

In [42]:
keys = [
    'System', 
    'Carrier', 
    'Output Type',
    'Conceptual Model', 
    'Abstraction Mechanism', 
    'Source', 
    'Language Form',
    'Coded Domain', 
    'Execution Model', 
    'Alt API Available', 
    'Extensible',
    'Formal Definition Available', 
    'Language', 
    'Data manipulation',
    'Provides Accessibility', 
    'Juxtaposition strategy', 
    'Allowed Data Type',
    'Data model', 
    'Interaction source', 
    'Open Source', 
    'Dependent',
    'Mark Types', 
    'Series Types', 
    'Output Type Coded',
    'Embedded language', 
    'Coordinate Systems'
    ]


binary_columns = [
    'Abstraction Mechanism',  
    'Alt API Available', 
    'Extensible',
    'Formal Definition Available', 
    'Provides Accessibility', 
    'Open Source', 
    'Dependent',
]

dumb_cols = [
    'Carrier', 
    'Output Type',
    'Conceptual Model',
    'Source', 
    'Language Form',
    'Coded Domain', 
    'Execution Model',
    'Language', 
    'Data manipulation',
    'Juxtaposition strategy', 
    'Allowed Data Type',
    'Data model', 
    'Interaction source', 
    'Output Type Coded',
    'Embedded language', 
]

split_cols = [
    'Mark Types', 
    # 'Series Types', 
    'Coordinate Systems'
]

df = data[keys].copy()

gen_split_cols = []
for col in split_cols:
    col_types = set(",".join(df[col].dropna().tolist()).split(','))
    for col_type in col_types:
        new_col = str(col + col_type + 'new')
        gen_split_cols.append(new_col)
        df[new_col] = df[col].apply(lambda x : 1 if col_type in str(x) and str(x) != 'nan' else 0)


for bin_col in binary_columns: 
    df[bin_col] = df[bin_col].apply(lambda x : 0 if "no" in x.lower() else 0)
    

keep_cols = ['System'] + dumb_cols + binary_columns + gen_split_cols
out_df = pd.get_dummies(df[keep_cols], columns=dumb_cols)
ana_df = out_df.copy().drop(['System'], axis=1)

In [157]:
def simple_plot(inp, color):
    local_df = pd.DataFrame(inp, columns=df['System']).T.reset_index().rename(columns={0: "x", 1: 'y'}).merge(df, on=['System'])
    base = alt.Chart(local_df).encode(
        x=alt.X("x", scale=alt.Scale(zero=False)), 
        y=alt.Y("y", scale=alt.Scale(zero=False)),
        color=alt.Color(color, scale=alt.Scale(range=["#EE0000", "#AF00DB", "#267f99"])),
        tooltip=["System"])
        # .properties(width=900)
    circles = base.mark_circle()
    txts = base.mark_text(dy=-10).encode(text="System")
    return circles + txts

In [44]:
pca = PCA(n_components=2)
pca.fit(ana_df.T)

PCA(n_components=2)

In [45]:
simple_plot(pca.components_, "Conceptual Model")

alt.LayerChart(...)

In [145]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(ana_df)

In [158]:
simple_plot(embedding.T, "Conceptual Model")

alt.LayerChart(...)

In [151]:
rel_data=pd.read_csv('./relationships.tsv',sep='\t')#.melt(id_vars=["System"])
subs = []
for idx in range(3):
    entity = "Rel " + str(idx + 1) + ": Entity"
    verb = "Rel " + str(idx + 1) + ": Verb"
    subs.append(rel_data.copy().rename(columns={entity: "Entity", verb: "Verb"})[['System', 'Entity', 'Verb']])
rel_data = pd.concat(subs).dropna()
local_df = pd.DataFrame(embedding.T, columns=df['System']).T.reset_index().rename(columns={0: "x", 1: 'y'}).merge(df, on=['System'])

coords = rel_data.merge(local_df[['x', 'y', 'System']], left_on="Entity", right_on="System").merge(local_df[['x', 'y', 'System']], left_on="System_x", right_on="System")
coord_df = coords[['Verb', 'x_x', 'y_x', 'x_y', 'y_y']].rename(columns={"x_x": "x1", "y_x": "y1", "x_y": "x2", "y_y": "y2"})
coord_df.reset_index(inplace=True)
coord_list = coord_df.rename(columns = {'index':'rel'}).to_dict('records') 
coord_return = []
for row in coord_list:
    coord_return.append({"verb": row['Verb'], "rel": row['rel'], "x": row['x1'], "y": row['y1']})
    coord_return.append({"verb": row['Verb'], "rel": row['rel'], "x": row['x2'], "y": row['y2']})
coord_df = pd.DataFrame(coord_return)

In [156]:


local_df = pd.DataFrame(embedding.T, columns=df['System']).T.reset_index().rename(columns={0: "x", 1: 'y'}).merge(df, on=['System'])
base = alt.Chart(local_df).encode(
    x=alt.X("x", scale=alt.Scale(zero=False)), 
    y=alt.Y("y", scale=alt.Scale(zero=False)),
    color=alt.Color("Conceptual Model", scale=alt.Scale(range=["#EE0000", "#AF00DB", "#267f99", "#0000ff", "#d16969", "#795E26"])),
    tooltip=["System"])\
    .properties(width=900)
circles = base.mark_circle()
txts = base.mark_text(dy=-10).encode(text="System")

lines = alt.Chart(coord_df).mark_line().encode(x="x", y="y", detail="rel", strokeDash="verb")

lines + circles + txts

alt.LayerChart(...)